Below, I install my own implementation of Professor Boonstra's "memoize DataFrame to disk" feature. The source code can be found at [github.com/ethho/memoize](https://github.com/ethho/memoize).

In [1]:
!python3 -m pip install git+https://github.com/ethho/memoize.git

  Cloning https://github.com/ethho/memoize.git to /tmp/pip-req-build-gd966lai
  Running command git clone --filter=blob:none --quiet https://github.com/ethho/memoize.git /tmp/pip-req-build-gd966lai
  Resolved https://github.com/ethho/memoize.git to commit bef633bd22e4acde44cccb63399a176c6cef79b9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import json
import re
import os
from glob import glob
from dataclasses import dataclass
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import quandl
import functools
import plotly.express as px
import plotly.graph_objects as go
from src.ubacktester import (
    BacktestEngine, StrategyBase, PositionBase, FeedBase,
    PlotlyPlotter, FeedID, PriceFeed, px_plot
)
from memoize.dataframe import memoize_df

%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format

# 20230126_hw3_ho_ethan_12350006

@mpcs
@finm33550

Ethan Ho 1/20/2023

----


## Configuration & Helper Functions

The following cell contains helper functions and configuration options that I will use in this notebook.

In [3]:
def get_secrets(fp='./secrets.json'):
    """
    Reads secret values such as API keys from a JSON-formatted file at `fp`.
    """
    with open(fp, 'r') as f:
        data = json.load(f)
    return data

def get_quandl_api_key() -> str:
    """
    Returns Quandl API key stored in secrets.json.
    """
    secrets = get_secrets()
    key = secrets.get('NASTAQ_DATA_API_KEY')
    assert key, f"NASTAQ_DATA_API_KEY field in secrets.json is empty or does not exist"
    return key

@memoize_df(cache_dir='/tmp/memoize')
def fetch_quandl_quotemedia_prices(
    start_date, end_date, ticker
) -> pd.DataFrame:
    df = quandl.get_table(
        'QUOTEMEDIA/PRICES',
        date={'gte': start_date, 'lte': end_date},
        ticker=ticker,
        api_key=get_quandl_api_key(),
    )
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', inplace=True)
    return df

@memoize_df(cache_dir='/tmp/memoize')
def fetch_quandl_tbill_prices(
    start_date, end_date,
) -> pd.DataFrame:
    """Fetch table of treasury bill prices from Quandl."""
    df = quandl.get(
        ['USTREASURY/BILLRATES'],
        returns="pandas",
        start_date=start_date,
        end_date=end_date,
        ticker=ticker,
        api_key=get_quandl_api_key(),
    )
    df = df.reset_index().rename(columns={'Date': 'date'})
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', inplace=True)
    return df

def risk_free_rate(**kw) -> float:
    """Calculates risk-free rate R_f from the 3-month T-bill rate."""
    tbill_prices = fetch_quandl_tbill_prices(**kw)
    tbill_returns = tbill_prices['USTREASURY/BILLRATES - 13 Wk Coupon Equiv']
    return tbill_returns.mean()

# Fetch Data Tables from Quandl

First, let's fetch the Zacks Fundmentals B. I chose to download zip archives as documented in the [Quandl API docs](https://github.com/quandl/quandl-python/blob/master/FOR_DEVELOPERS.md#datatable).

In [4]:
! mkdir -p data
for table in (
    'FC', 'FR', 
    'MT', 'HDM',
    'MKTV', 'SHRS', 
):
    table_lower = table.lower()
    zip_fp = f'data/zacks_{table.lower()}.zip'
    if os.path.isfile(zip_fp):
        print(f"Zip file {zip_fp} already exists. Skipping download")
    else:
        export_table_kwargs = dict(
            filename=zip_fp,
            api_key=get_quandl_api_key(),
        )
        if table not in ('MT', 'HDM'):
            export_table_kwargs['per_end_date'] = {
                'gte': "2015-01-01", 'lte': "2022-01-01"
            }
        fp = quandl.export_table(f'ZACKS/{table}',**export_table_kwargs)
        print(f"Wrote ZIP file to {zip_fp}")
    ! unzip -o -d data/zacks_{table_lower} {zip_fp}

Zip file data/zacks_fc.zip already exists. Skipping download
Archive:  data/zacks_fc.zip
  inflating: data/zacks_fc/ZACKS_FC_2_74d39c53d83a9fe6b173451893cce9cf.csv  
Zip file data/zacks_fr.zip already exists. Skipping download
Archive:  data/zacks_fr.zip
  inflating: data/zacks_fr/ZACKS_FR_2_a1fbee771fb67f7d385fb5fc1c747432.csv  
Zip file data/zacks_mt.zip already exists. Skipping download
Archive:  data/zacks_mt.zip
  inflating: data/zacks_mt/ZACKS_MT_2_5c2afb6368dcc3ed48e1a84279323e63.csv  
Zip file data/zacks_hdm.zip already exists. Skipping download
Archive:  data/zacks_hdm.zip
  inflating: data/zacks_hdm/ZACKS_HDM_2_5035a21ce4e45c8e7fbda0facdcfcb6c.csv  
Zip file data/zacks_mktv.zip already exists. Skipping download
Archive:  data/zacks_mktv.zip
  inflating: data/zacks_mktv/ZACKS_MKTV_2_76cabe81a35ac38909e4101cceb896bb.csv  
Zip file data/zacks_shrs.zip already exists. Skipping download
Archive:  data/zacks_shrs.zip
  inflating: data/zacks_shrs/ZACKS_SHRS_2_8327b1978811ad1d9013352

The next command will check file sizes, to make sure that we're not occupying too much disk space on graders' machines.

In [5]:
! du -hs data/zacks_*

277M	data/zacks_fc
72M	data/zacks_fc.zip
57M	data/zacks_fr
16M	data/zacks_fr.zip
3.3M	data/zacks_hdm
1.1M	data/zacks_hdm.zip
22M	data/zacks_mktv
2.8M	data/zacks_mktv.zip
3.8M	data/zacks_mt
972K	data/zacks_mt.zip
22M	data/zacks_shrs
2.2M	data/zacks_shrs.zip


Here's an example of how to load one of the data tables (SHRS) into memory using pandas:

In [6]:
shrs = pd.read_csv(*glob('data/zacks_shrs/*.csv'))
shrs.head()

,ticker,m_ticker,comp_name,fye,per_type,per_end_date,active_ticker_flag,shares_out,avg_d_shares
0,A,A2,AGILENT TECH,10,Q,2021-10-31,Y,302.0000,305.0000
1,A,A2,AGILENT TECH,10,Q,2021-07-31,Y,302.7200,306.0000
2,A,A2,AGILENT TECH,10,Q,2021-04-30,Y,303.4400,307.0000
3,A,A2,AGILENT TECH,10,Q,2021-01-31,Y,304.7000,309.0000
4,A,A2,AGILENT TECH,10,Q,2020-10-31,Y,308.3100,311.0000


# From HW 3 Prompt

Find $\ge$ 200 tickers where the following conditions are met for our analysis period of 1/2015 - 1/2022:

- Not in automotive, financial, or insurance sector at any point in the period
    - See `FC/ZACKS_X_IND_CODE`, `FC/ZACKS_SECTOR_CODE`, and the [classification list](http://www.zacksdata.com/app/download/247340904/Zacks+Sector+Industry+Mapping+Scheme.pdf) (and maybe `FC/ZACKS_METRICS_IND_CODE` too?)
- Debt/market cap ratio is $>0.1$ for some nontrivial amount of time. Should be about 1000-2000 companies including ASH, VTOL, ISUN, and VIVO.
- Calculation of the following ratios is feasible:
    - Debt to market cap
        - See `FR/TOT_DEBT_TOT_EQUITY`
    - Return on investment (ROI)
        - See `FR/RET_INVST`, `MKTV/MKT_VAL`, `FC/NET_LTERM_DEBT`, `FC/TOT_LTERM_DEBT`
    - Price to earnings (P/E)
        - See `FC/EPS_DILUTED_NET`, `FC/BASIC_NET_EPS`, `SHRS/SHARES_OUT`, `MKTV/MKT_VAL`, `GAAP`
        
# To-Do

- Strip string dtypes
- ffill trading day to `per_end_date` from previous trading day
- Narrow data query to only the columns we use below

# Data Munging to Get List of Tickers

From the FC table, let's filter out tickers in the excluded sectors. By the way, most of the time, when I refer to ticker, I'm actually referring to the `m_ticker`, which is easier to track historically as securities switch between exchanges.

In [7]:
fc = pd.read_csv(*glob('data/zacks_fc/*.csv'), low_memory=True)

# Set int dtype for industry codes
fc['zacks_sector_code'] = fc['zacks_sector_code'].fillna(-1.).astype(int)
fc['zacks_x_ind_code'] = fc['zacks_x_ind_code'].fillna(-1.).astype(int)

# Set MultiIndex of date and m_ticker
fc['per_end_date'] = pd.to_datetime(fc['per_end_date'])
fc.sort_values(by='per_end_date')
fc.set_index(['m_ticker', 'per_end_date'], inplace=True)
fc.head()

/tmp/ipykernel_2839/2272979075.py:1: DtypeWarning: Columns (4,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,44,45,46,47,49,50,51,52,53,54,55,56,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  fc = pd.read_csv(*glob('data/zacks_fc/*.csv'), low_memory=True)


ticker comp_name     comp_name_2 exchange currency_code  \
m_ticker per_end_date                                                           
CPG      2015-03-31    3262Q       NaN   CMS ENERGY -G      NaN           USD   
         2015-06-30    3262Q       NaN   CMS ENERGY -G      NaN           USD   
FML      2015-12-31    3346Q       NaN  US BANCORP-OLD      NaN           USD   
         2016-12-31    3346Q       NaN  US BANCORP-OLD      NaN           USD   
         2017-12-31    3346Q       NaN  US BANCORP-OLD      NaN           USD   

                      per_type  per_code  per_fisc_year  per_fisc_qtr  \
m_ticker per_end_date                                                   
CPG      2015-03-31          Q       NaN           2015             1   
         2015-06-30          Q       NaN           2015             2   
FML      2015-12-31          A       NaN           2015             4   
         2016-12-31          A       NaN           2016             4   
         2017-12-31          A       NaN           2017             4   

                       per_cal_year  ...  stock_based_compsn_qd  \
m_ticker per_end_date                ...                          
CPG      2015-03-31            2015  ...                    NaN   
         2015-06-30            2015  ...                    NaN   
FML      2015-12-31            2015  ...                    NaN   
         2016-12-31            2016  ...                    NaN   
         2017-12-31            2017  ...                    NaN   

                      cash_flow_oper_activity_qd  \
m_ticker per_end_date                              
CPG      2015-03-31                     760.0000   
         2015-06-30                     502.0000   
FML      2015-12-31                          NaN   
         2016-12-31                          NaN   
         2017-12-31                          NaN   

                      net_change_prop_plant_equip_qd  comm_stock_div_paid_qd  \
m_ticker per_end_date                                                          
CPG      2015-03-31                        -348.0000                     NaN   
         2015-06-30                        -327.0000                     NaN   
FML      2015-12-31                              NaN                     NaN   
         2016-12-31                              NaN                     NaN   
         2017-12-31                              NaN                     NaN   

                       pref_stock_div_paid_qd  tot_comm_pref_stock_div_qd  \
m_ticker per_end_date                                                       
CPG      2015-03-31                       NaN                    -80.0000   
         2015-06-30                       NaN                    -81.0000   
FML      2015-12-31                       NaN                         NaN   
         2016-12-31                       NaN                         NaN   
         2017-12-31                       NaN                         NaN   

                       wavg_shares_out  wavg_shares_out_diluted eps_basic_net  \
m_ticker per_end_date                                                           
CPG      2015-03-31                NaN                      NaN           NaN   
         2015-06-30                NaN                      NaN           NaN   
FML      2015-12-31         1,764.0000               1,772.0000        3.1800   
         2016-12-31         1,718.0000               1,724.0000        3.2500   
         2017-12-31         1,677.0000               1,683.0000        3.5300   

                      eps_diluted_net  
m_ticker per_end_date                  
CPG      2015-03-31               NaN  
         2015-06-30               NaN  
FML      2015-12-31            3.1600  
         2016-12-31            3.2400  
         2017-12-31            3.5100  

[5 rows x 247 columns]

We define which sectors we want to filter out for each column:

In [8]:
exclude_codes = {
    'zacks_sector_code': (
        5, # automotive
        13, # finance
    ),
    'zacks_x_ind_code': [
        7, 8, 9, 10, 11, 210, # automotive
        *range(61, 70), # finance
        *range(85, 90), # insurance
    ]
}

In [9]:
print(f"There are {fc.index.get_level_values(0).unique().size} unique tickers before sector filtering.")

There are 11698 unique tickers before sector filtering.


In [10]:
for col, exclude_vals in exclude_codes.items():
    fc = fc[~fc[col].isin(exclude_vals)]

In [11]:
print(f"There are {fc.index.get_level_values(0).unique().size} unique tickers after sector filtering.")

There are 9524 unique tickers after sector filtering.


We removed about $\frac{1}{5}$ of the tickers with this filter.

Next, we'll filter to tickers where debt/market cap $> 1$. To do this, we need to merge in `FR/TOT_DEBT_TOT_EQUITY` and `MKTV/MKT_VAL`.

In [12]:
fr = pd.read_csv(*glob('data/zacks_fr/*.csv'))
fr['per_end_date'] = pd.to_datetime(fr['per_end_date'])
fr.sort_values(by='per_end_date')
fr.set_index(['m_ticker', 'per_end_date'], inplace=True)
fr.head()

ticker comp_name     comp_name_2 exchange currency_code  \
m_ticker per_end_date                                                           
CPG      2015-03-31    3262Q       NaN   CMS ENERGY -G      NaN           USD   
         2015-06-30    3262Q       NaN   CMS ENERGY -G      NaN           USD   
FML      2015-12-31    3346Q       NaN  US BANCORP-OLD      NaN           USD   
         2016-12-31    3346Q       NaN  US BANCORP-OLD      NaN           USD   
         2017-12-31    3346Q       NaN  US BANCORP-OLD      NaN           USD   

                      per_type  per_code  per_fisc_year  per_fisc_qtr  \
m_ticker per_end_date                                                   
CPG      2015-03-31          Q       NaN           2015             1   
         2015-06-30          Q       NaN           2015             2   
FML      2015-12-31          A       NaN           2015             4   
         2016-12-31          A       NaN           2016             4   
         2017-12-31          A       NaN           2017             4   

                       per_cal_year  ...  invty_turn  rcv_turn  day_sale_rcv  \
m_ticker per_end_date                ...                                       
CPG      2015-03-31            2015  ...         NaN       NaN           NaN   
         2015-06-30            2015  ...         NaN       NaN           NaN   
FML      2015-12-31            2015  ...         NaN       NaN           NaN   
         2016-12-31            2016  ...         NaN       NaN           NaN   
         2017-12-31            2017  ...         NaN       NaN           NaN   

                       ret_equity  ret_tang_equity  ret_asset  ret_invst  \
m_ticker per_end_date                                                      
CPG      2015-03-31           NaN              NaN        NaN        NaN   
         2015-06-30       -0.0208          -0.0093    -0.0055    -0.0203   
FML      2015-12-31       14.3601          17.3958     1.4064     7.5201   
         2016-12-31       14.0083          16.8452     1.3328     7.3152   
         2017-12-31       14.1320          16.8982     1.3533     7.6326   

                       free_cash_flow_per_share  book_val_per_share  \
m_ticker per_end_date                                                 
CPG      2015-03-31                         NaN                 NaN   
         2015-06-30                         NaN                 NaN   
FML      2015-12-31                      4.9560             26.8263   
         2016-12-31                      3.0951             28.2472   
         2017-12-31                      3.8455             29.9980   

                       oper_cash_flow_per_share  
m_ticker per_end_date                            
CPG      2015-03-31                         NaN  
         2015-06-30                         NaN  
FML      2015-12-31                      4.9560  
         2016-12-31                      3.0951  
         2017-12-31                      3.8455  

[5 rows x 38 columns]

In [13]:
print(f"There are {fr.index.get_level_values(0).unique().size} unique tickers in the FR table.")

There are 11698 unique tickers in the FR table.


In [14]:
mktv = pd.read_csv(*glob('data/zacks_mktv/*.csv'))
mktv['per_end_date'] = pd.to_datetime(mktv['per_end_date'])
mktv.sort_values(by='per_end_date')
mktv.set_index(['m_ticker', 'per_end_date'], inplace=True)
mktv.head()

ticker     comp_name  fye per_type active_ticker_flag  \
m_ticker   per_end_date                                                         
A2         2021-12-31        A  AGILENT TECH   10        Q                  Y   
           2021-09-30        A  AGILENT TECH   10        Q                  Y   
           2021-06-30        A  AGILENT TECH   10        Q                  Y   
           2021-03-31        A  AGILENT TECH   10        Q                  Y   
           2020-12-31        A  AGILENT TECH   10        Q                  Y   

                            mkt_val      ep_val  
m_ticker   per_end_date                          
A2         2021-12-31   47,953.4500 49,107.4500  
           2021-09-30   47,687.9000 48,987.9000  
           2021-06-30   44,851.9500 46,198.9500  
           2021-03-31   38,739.2400 39,595.2400  
           2020-12-31   36,185.5800 37,028.5800

In [15]:
print(f"There are {fr.index.get_level_values(0).unique().size} unique tickers in the MKTV table.")

There are 11698 unique tickers in the MKTV table.


TODO: will a date that's in the right df but not the left persist in a left merge?

In [26]:
fr_mkt = fr.merge(mktv['mkt_val'], how='outer', left_index=True, right_index=True)
fr_mkt

ticker                comp_name  \
m_ticker per_end_date                                   
#AAO     2015-03-31     AAON                     AAON   
         2015-06-30     AAON                     AAON   
         2015-09-30     AAON                     AAON   
         2015-12-31     AAON                     AAON   
         2015-12-31     AAON                     AAON   
...                      ...                      ...   
NaN      2020-01-31     T.NA  National Bank of Canada   
         2020-04-30     T.NA  National Bank of Canada   
         2020-07-31     T.NA  National Bank of Canada   
         2020-10-31     T.NA  National Bank of Canada   
         2020-10-31     T.NA  National Bank of Canada   

                                   comp_name_2 exchange currency_code  \
m_ticker per_end_date                                                   
#AAO     2015-03-31                 AAON, Inc.   NASDAQ           USD   
         2015-06-30                 AAON, Inc.   NASDAQ           USD   
         2015-09-30                 AAON, Inc.   NASDAQ           USD   
         2015-12-31                 AAON, Inc.   NASDAQ           USD   
         2015-12-31                 AAON, Inc.   NASDAQ           USD   
...                                        ...      ...           ...   
NaN      2020-01-31    National Bank of Canada      NaN           CND   
         2020-04-30    National Bank of Canada      NaN           CND   
         2020-07-31    National Bank of Canada      NaN           CND   
         2020-10-31    National Bank of Canada      NaN           CND   
         2020-10-31    National Bank of Canada      NaN           CND   

                      per_type  per_code  per_fisc_year  per_fisc_qtr  \
m_ticker per_end_date                                                   
#AAO     2015-03-31          Q       NaN     2,015.0000        1.0000   
         2015-06-30          Q       NaN     2,015.0000        2.0000   
         2015-09-30          Q       NaN     2,015.0000        3.0000   
         2015-12-31          A       NaN     2,015.0000        4.0000   
         2015-12-31          Q       NaN     2,015.0000        4.0000   
...                        ...       ...            ...           ...   
NaN      2020-01-31          Q       NaN     2,020.0000        1.0000   
         2020-04-30          Q       NaN     2,020.0000        2.0000   
         2020-07-31          Q       NaN     2,020.0000        3.0000   
         2020-10-31          A       NaN     2,020.0000        4.0000   
         2020-10-31          Q       NaN     2,020.0000        4.0000   

                       per_cal_year  ...  rcv_turn  day_sale_rcv  ret_equity  \
m_ticker per_end_date                ...                                       
#AAO     2015-03-31      2,015.0000  ...    1.8172       49.5277      4.6118   
         2015-06-30      2,015.0000  ...    1.8474       48.7161      5.9382   
         2015-09-30      2,015.0000  ...    1.9660       45.7793      6.6799   
         2015-12-31      2,015.0000  ...    6.5532       55.6977     25.5581   
         2015-12-31      2,015.0000  ...    1.7767       50.6571      7.2368   
...                             ...  ...       ...           ...         ...   
NaN      2020-01-31      2,020.0000  ...       NaN           NaN      4.7028   
         2020-04-30      2,020.0000  ...       NaN           NaN      2.8360   
         2020-07-31      2,020.0000  ...       NaN           NaN      4.6127   
         2020-10-31      2,020.0000  ...       NaN           NaN     15.5066   
         2020-10-31      2,020.0000  ...       NaN           NaN      3.6626   

                       ret_tang_equity  ret_asset  ret_invst  \
m_ticker per_end_date                                          
#AAO     2015-03-31             4.6118     3.5475     4.6118   
         2015-06-30             5.9382     4.4518     5.9382   
         2015-09-30             6.6799     5.1945     6.6799   
         2015-12-31

In [17]:
def idx_diff(df1, df2):
    return set(df1.index.get_level_values(1).unique()).difference(df2.index.get_level_values(1).unique())

In [42]:
mktv[mktv.ticker == 'AAPL'].index

MultiIndex([('AAPL      ', '2021-12-31'),
            ('AAPL      ', '2021-09-30'),
            ('AAPL      ', '2021-06-30'),
            ('AAPL      ', '2021-03-31'),
            ('AAPL      ', '2020-12-31'),
            ('AAPL      ', '2020-09-30'),
            ('AAPL      ', '2020-06-30'),
            ('AAPL      ', '2020-03-31'),
            ('AAPL      ', '2019-12-31'),
            ('AAPL      ', '2019-09-30'),
            ('AAPL      ', '2019-06-30'),
            ('AAPL      ', '2019-03-31'),
            ('AAPL      ', '2018-12-31'),
            ('AAPL      ', '2018-09-30'),
            ('AAPL      ', '2018-06-30'),
            ('AAPL      ', '2018-03-31'),
            ('AAPL      ', '2017-12-31'),
            ('AAPL      ', '2017-09-30'),
            ('AAPL      ', '2017-06-30'),
            ('AAPL      ', '2017-03-31'),
            ('AAPL      ', '2016-12-31'),
            ('AAPL      ', '2016-09-30'),
            ('AAPL      ', '2016-06-30'),
            ('AAPL      ', '2016-0

In [38]:
fr_mkt.loc['AAPL']

,ticker,comp_name,comp_name_2,exchange,currency_code,per_type,per_code,per_fisc_year,per_fisc_qtr,per_cal_year,...,rcv_turn,day_sale_rcv,ret_equity,ret_tang_equity,ret_asset,ret_invst,free_cash_flow_per_share,book_val_per_share,oper_cash_flow_per_share,mkt_val
per_end_date,,,,,,,,,,,,,,,,,,,,,
2015-03-31,AAPL,Apple,Apple Inc.,NASDAQ,USD,Q,NaN,"2,015.0000",2.0000,"2,015.0000",...,3.1937,28.1807,10.5181,11.2855,5.1950,8.0253,2.0231,5.5970,2.2624,NaN
2015-06-30,AAPL,Apple,Apple Inc.,NASDAQ,USD,Q,NaN,"2,015.0000",3.0000,"2,015.0000",...,2.4918,36.1179,8.4956,9.1370,3.9088,6.1683,2.6053,5.5069,2.9356,NaN
2015-09-30,AAPL,Apple,Apple Inc.,NASDAQ,USD,A,NaN,"2,015.0000",4.0000,"2,015.0000",...,7.7024,47.3876,44.7355,48.3878,18.3899,30.9201,3.0217,5.3486,3.5070,NaN
2015-09-30,AAPL,Apple,Apple Inc.,NASDAQ,USD,Q,NaN,"2,015.0000",4.0000,"2,015.0000",...,1.6973,53.0256,9.3201,10.0810,3.8313,6.4418,3.0217,5.3486,3.5070,NaN
2015-12-31,AAPL,Apple,Apple Inc.,NASDAQ,USD,Q,NaN,"2,016.0000",1.0000,"2,015.0000",...,3.0816,29.2056,14.3147,15.4112,6.2605,10.1179,1.0659,5.7835,1.2273,NaN
2016-03-31,AAPL,Apple,Apple Inc.,NASDAQ,USD,Q,NaN,"2,016.0000",2.0000,"2,016.0000",...,2.5503,35.2901,8.0609,8.6648,3.4447,5.2624,1.4942,5.9532,1.7625,NaN
2016-06-30,AAPL,Apple,Apple Inc.,NASDAQ,USD,Q,NaN,"2,016.0000",3.0000,"2,016.0000",...,2.2245,40.4594,6.1608,6.6195,2.5510,3.9881,1.8702,5.8660,2.2702,NaN
2016-09-30,AAPL,Apple,Apple Inc.,NASDAQ,USD,A,NaN,"2,016.0000",4.0000,"2,016.0000",...,7.3599,49.5928,35.6237,38.1906,14.2024,22.4312,2.4316,6.0085,3.0103,NaN
2016-09-30,AAPL,Apple,Apple Inc.,NASDAQ,USD,Q,NaN,"2,016.0000",4.0000,"2,016.0000",...,1.5991,56.2817,7.0285,7.5350,2.8021,4.4257,2.4316,6.0085,3.0103,NaN
